# Messages
Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Basic Usage

In [8]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-5-nano")


In [9]:
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    tools=[],
    prompt="You are a full-stack comedian"
)

In [4]:
human_msg = HumanMessage("Hello, how are you?")

result = agent.invoke({"messages": [human_msg]})

In [5]:
print(result["messages"][-1].content)

Hey there! I’m doing great—fully loaded, cache-friendly, and ready to serve up some laughs. How about you? Want a quick tech joke or is there something you’d like help with today?


In [6]:
print(type(result["messages"][-1]))

<class 'langchain_core.messages.ai.AIMessage'>


In [7]:
for msg in result["messages"]:
    print(f"{msg.type}: {msg.content}\n")

human: Hello, how are you?

ai: Hey there! I’m doing great—fully loaded, cache-friendly, and ready to serve up some laughs. How about you? Want a quick tech joke or is there something you’d like help with today?



> question to Sydney?: Why don't I see the system message here?

### Altenative formats
#### Strings

In [12]:
agent = create_agent(
    model=model,
    tools=[],
    prompt="You are a terse sports poet."  # This is a SystemMessage under the hood
)

In [14]:
result = agent.invoke({"messages": "Tell me about baseball"})
print(result["messages"][-1].content)

Baseball: a diamond cut in chalk and sun.
Nine innings breathe; three outs pace the drum.
A pitcher fires heat, a curve bends the air.
The batter hunts the sweet spot, the bat sings clean.
Gloves bloom in the grass, the infield tells its tale.
Catcher crouched, eyes twins of a vigil at the plate.
Umpire’s call—strike, ball, safe, out—fate in white.
Bases as a heartbeat, the crowd riding the next pitch.


#### Dictionaries

In [12]:
result = agent.invoke(
    {"messages": {"role": "user", "content": "Write a haiku about sprinters"}}
)
print(result["messages"][-1].content)

Lunging at the line
Every breath a mirrored wind
Time loosens its hold


There are multiple roles:
```python
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]
```

## Output Format
### messages

In [43]:
from langchain_core.tools import tool
@tool
def check_haiku_lines(text: str):
    """
    Checks if the given haiku text has exactly 3 lines.
    Returns None if it's correct, otherwise an error message.
    """
    # Split the text into lines, ignoring leading/trailing spaces
    lines = [line.strip() for line in text.strip().splitlines() if line.strip()]
    print(f"checking haiku, it has {len(lines)} lines:\n {text}")

    if len(lines) != 3:
        return f"Incorrect! This haiku has {len(lines)} lines. A haiku must have exactly 3 lines."
    return "Correct, This haiku has 3 lines."


In [50]:
agent = create_agent(
    model="openai:gpt-5",
    tools=[check_haiku_lines],
    prompt="You are a sports poet who only writes Haiku. You always check your work."  
)

In [51]:
result = agent.invoke({"messages" :"Please write me a poem"})

checking haiku, it has 3 lines:
 Whistle splits the dusk
Footsteps drum on frosted grass
Dreams race past the posts


In [53]:
result["messages"][-1].content

'Whistle splits the dusk\nFootsteps drum on frosted grass\nDreams race past the posts'

In [60]:
print(len(result["messages"]))

4


In [71]:
for i, msg in enumerate(result["messages"]):
    msg.pretty_print()
    #print(f"{msg.type}: content:{msg.content} tool_call:{msg.content}\n")

================================ Human Message =================================

Please write me a poem
================================== Ai Message ==================================
Tool Calls:
  check_haiku_lines (call_LFp9uarPPtWHeEInSPqCOQWY)
 Call ID: call_LFp9uarPPtWHeEInSPqCOQWY
  Args:
    text: Whistle splits the dusk
Footsteps drum on frosted grass
Dreams race past the posts
================================= Tool Message =================================
Name: check_haiku_lines

Correct, This haiku has 3 lines.
================================== Ai Message ==================================

Whistle splits the dusk
Footsteps drum on frosted grass
Dreams race past the posts


### Other useful information

In [64]:
result

{'messages': [HumanMessage(content='Please write me a poem', additional_kwargs={}, response_metadata={}, id='b0ac5bdb-8d7a-45b0-97ba-d51c60f53450'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 493, 'prompt_tokens': 170, 'total_tokens': 663, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CN1hcXRtOI8CJ0YG4F8QTH3q2ZkNj', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--1054fee7-d98c-463a-9aec-26949cfa28aa-0', tool_calls=[{'name': 'check_haiku_lines', 'args': {'text': 'Whistle splits the dusk\nFootsteps drum on frosted grass\nDreams race past the posts'}, 'id': 'call_LFp9uarPPtWHeEInSPqCOQWY', 'type': 'tool_call'}], 

In [75]:
result["messages"][-1]

AIMessage(content='Whistle splits the dusk\nFootsteps drum on frosted grass\nDreams race past the posts', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 233, 'total_tokens': 256, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CN1hi0yKRDdLhLy8Dj1XhfESX4wbX', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--e5e94e60-1588-4e9f-b0a9-fbc507e50a97-0', usage_metadata={'input_tokens': 233, 'output_tokens': 23, 'total_tokens': 256, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [78]:
result["messages"][-1].usage_metadata

{'input_tokens': 233,
 'output_tokens': 23,
 'total_tokens': 256,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 0}}

In [77]:
result["messages"][-1].response_metadata

{'token_usage': {'completion_tokens': 23,
  'prompt_tokens': 233,
  'total_tokens': 256,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_provider': 'openai',
 'model_name': 'gpt-5-2025-08-07',
 'system_fingerprint': None,
 'id': 'chatcmpl-CN1hi0yKRDdLhLy8Dj1XhfESX4wbX',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

# OLD

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

model = init_chat_model("openai:gpt-5-nano")

system_msg = SystemMessage("You are a helpful assistant.")
human_msg = HumanMessage("Hello, how are you?")

# Use with chat models
messages = [system_msg, human_msg]
response = model.invoke(messages)  # Returns AIMessage

In [ ]:
print(type(response))

In [ ]:
print(response.text)

## Formats

### Text Prompt
Text prompts are strings - ideal for straightforward generation tasks where you don’t need to retain conversation history.

In [ ]:
response = model.invoke("Write a haiku about spring")

In [ ]:
print(response.text)

### Message Prompts

In [ ]:
system_msg = SystemMessage("You are a helpful sports poet")
ai_msg = AIMessage("You're right, my rhymes are sublime.")
human_msg = HumanMessage("Write a haiku about baseball")
messages = [system_msg, ai_msg, human_msg]

In [ ]:
response = model.invoke(messages)

In [ ]:
print(response.text)

#### An alternate form of messages is a dictionary format.

In [ ]:
messages = [
    {"role": "system", "content": "You are a sports poetry expert who completes haikus that have been started"},
    {"role": "user", "content": "Write a haiku about sprinters"},
    {"role": "assistant", "content": "Feet don't fail me..."}
]

In [ ]:
response = model.invoke(messages)

In [ ]:
print(response.text)

## Printing
You will often see 'pretty printing' in this course.

In [ ]:
system_msg = SystemMessage("You are a helpful sports poet")
print (system_msg)   # is system_msg.__str__()

In [ ]:
print(system_msg.pretty_print())

In [ ]:
print(system_msg.model_dump())  # all fields

In [ ]:
print(type(system_msg))

## Content

In [ ]:
from langchain_core.messages import HumanMessage

# String content
human_message = HumanMessage("Hello, how are you?")

# Provider-native format (e.g., OpenAI)
human_message = HumanMessage(content=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image_url", "image_url": {"url": "https://example.com/image.jpg"}}
])

# List of standard content blocks
human_message = HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://example.com/image.jpg"},
])

In [ ]:
from IPython.display import Image

# Just give it a file path
Image("./assets/Graphic.png",width=200)

In [ ]:
import base64
from langchain_core.messages import SystemMessage, HumanMessage

# Encode local image as base64
filename = "./assets/Graphic.png"
with open(filename, "rb") as f:
    image_bytes = f.read()
image_b64 = base64.b64encode(image_bytes).decode("utf-8")

# Messages
system_msg = SystemMessage("You are a helpful image reader!")
human_msg = HumanMessage(content=[
    {"type": "text", "text": "Please, tell me what this image means."},
    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}}
])

messages = [system_msg, human_msg]

# Call GPT-5 (vision-capable)
response = model.invoke(messages, model="gpt-5")

print(response.content)

## AIMessage fields
AI messages from an LLM contain several useful fields beyond the content field.

In [ ]:
response = model.invoke("Write a haiku about spring")
response

In [ ]:
response.content

In [ ]:
response.response_metadata

In [ ]:
response.usage_metadata

## Content

In [ ]:
from langchain_core.messages import HumanMessage

# String content
human_message = HumanMessage("Hello, how are you?")

# Provider-native format (e.g., OpenAI)
human_message = HumanMessage(content=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image_url", "image_url": {"url": "https://example.com/image.jpg"}}
])

# List of standard content blocks
human_message = HumanMessage(content_blocks=[
    {"type": "text", "text": "Hello, how are you?"},
    {"type": "image", "url": "https://example.com/image.jpg"},
])

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

model = init_chat_model("openai:gpt-5-nano")

In [ ]:
from IPython.display import Image

# Just give it a file path
Image("./assets/Graphic.png",width=200)

In [ ]:
import base64
from langchain_core.messages import SystemMessage, HumanMessage

# Encode local image as base64
filename = "./assets/Graphic.png"
with open(filename, "rb") as f:
    image_bytes = f.read()
image_b64 = base64.b64encode(image_bytes).decode("utf-8")

# Messages
system_msg = SystemMessage("You are a helpful image reader!")
human_msg = HumanMessage(content=[
    {"type": "text", "text": "Please, tell me what this image means."},
    {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{image_b64}"}}
])

messages = [system_msg, human_msg]

# Call GPT-5 (vision-capable)
response = model.invoke(messages, model="gpt-5")

print(response.content)

In [ ]:
### Content Blocks

In [ ]:
# Messages
system_msg = SystemMessage("You are a helpful image reader!")
human_msg = HumanMessage(content_blocks=[
    {"type": "text", "text": "Please, tell me what this image means."},
    {"type": "image", "base64": image_b64, "mime_type": "image/png"}
])

messages = [system_msg, human_msg]

# Call GPT-5 (vision-capable)
response = model.invoke(messages, model="gpt-5")

print(response.content)

In [ ]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

#llm = init_chat_model("claude-3-5-sonnet-latest", model_provider="anthropic")
llm = init_chat_model("gpt-5", model_provider="openai")

agent = create_agent(
    model=llm,
    tools=[],
    prompt=SystemMessage(content="You are a funny guy"),
)

In [ ]:
for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
result = agent.invoke(
    {"messages": [{"role": "user", 
                   "content": "Tell me a joke"}]}
    )
print(result["messages"][1])

In [ ]:
print(result["messages"][1])

In [ ]:
# Stream = values
for step in agent.stream(
    {"messages": [{"role": "user", 
                   "content": "Tell me a joke"}]},
    stream_mode="values",
    ):
    step["messages"][-1].pretty_print()

In [ ]:
for token, metadata in agent.stream(
    {"messages": [{"role": "user", "content": "What is the weather in SF?"}]},
    stream_mode="messages",
):
    print(f"node: {metadata['langgraph_node']}")
    print(f"content: {token.content_blocks}")
    print("\n")